# Tutorial: a multi-material simulation with optimism

In [1]:
import numpy as onp # as in "old" numpy, to distinguish it from Jax's numpy.

from optimism.JaxConfig import *
from optimism import EquationSolver
from optimism import FunctionSpace
from optimism import Interpolants
from optimism.material import J2Plastic
from optimism.material import Neohookean
from optimism import Mechanics
from optimism import Mesh
from optimism import Objective
from optimism import QuadratureRule
from optimism import SparseMatrixAssembler
from optimism import VTKWriter

## Overview of the problem

In this problem, we'll subject a rectangular slab to plane strain uniaxial tension. As shown in the figure, the slab is made of two equal-size layers made of different materials. One material will use a hyperelastic material model, while the other will use a $J_2$ plasticity model.

*Figure here*

## Set up the mesh

As in any finite element code, first we need to set up the geometry of the problem with a mesh. Optimism can read in meshes from the Exodus II format, but it also provides utilities for generating structured meshes for simple problems like this one. Let's generate a rectangular mesh with width `w` and length `L`. We'll also specify that it has 5 nodes in the $x$-direction (hence 4 elements) and 15 nodes in the $y$-direction.

In [2]:
# set up the mesh
w = 0.1
L = 1.0
nodesInX = 5 # must be odd
nodesInY = 15
xRange = [0.0, w]
yRange = [0.0, L]
mesh = Mesh.construct_structured_mesh(nodesInX, nodesInY, xRange, yRange)

The mesh object has an attribute called `blocks` which lets us group elements together to specify different materials on them. Let's see what our mesh has:

In [3]:
print(mesh.blocks)

{'block_0': DeviceArray([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
              12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,
              24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
              36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,
              48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
              60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
              72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
              84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
              96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107,
             108, 109, 110, 111], dtype=int64)}


`blocks` is a standard Python dictionary object (i.e. a `dict`), where the keys are strings that lets us give meaningful names to the element blocks. The values are Jax-numpy arrays that contain the indices of the elements in the block. The `construct_structured_mesh` function returns a mesh with just one block. We want two equal-sized blocks for our problem like in the figure, so let's modify the mesh.

First, we'll define a simple helper function that takes in the vertices of a triangular element and returns the coordinates of the centroid.

In [4]:
def triangle_centroid(vertices):
    return np.average(vertices, axis=0)

We'll loop over the element node connectivity table, which is the `mesh.conns` object, extract the coordinates of its vertices, and use the `triangle_centroid` object on them to determine if the element is in the left or right layer. We will store the results in a list called `blockIds`:

In [26]:
# initialize the block IDs of all elements to a dummy value (-1)
blockIds = -1*onp.ones(Mesh.num_elements(mesh), dtype=onp.int64)
for e, t in enumerate(mesh.conns):
    vertices = mesh.coords[t,:]
    if triangle_centroid(vertices)[0] < w/2:
        blockIds[e] = 0
    else:
        blockIds[e] = 1
# check that every element has gotten an ID
assert(onp.all(blockIds != -1))

This will mark an element as block 0 if it's centroid is on the left hand side of the slab, and as block 1 if it's on the other side. Now, let's make the `dict` that we want to attach to the mesh object.

In [6]:
blocks = {'left layer': np.flatnonzero(np.array(blockIds) == 0),
          'right layer': np.flatnonzero(np.array(blockIds) == 1)}

Now we can make use of a function that takes in the original mesh (with one block) and the block IDS we just created, and returns a new mesh that is the same as the old one except that the blocks have been updated.

In [7]:
mesh = Mesh.mesh_with_blocks(mesh, blocks)

Let's check to make sure this process worked. To do this, we'll make use of optimism's output facilities. Optimism provides a class called `VTKWriter`, that, as the name suggests, writes data to VTK files which can be visualized in ParaView (and several other visualization tools). First, we instantiate a VTKWriter object, giving it the base of the filename (the name that will be suffixed with the ".vtk" extension).

In [8]:
writer = VTKWriter.VTKWriter(mesh, baseFileName='check_problem_setup')

Next, we can start adding fields. In this case, we only have one field - the block ID numbers - which is a scalar field of integer type. Finally, once our data is loaded into the writer, we call the `write()` method to tell it to write the VTK file to disk.

In [9]:
writer.add_cell_field(name='block_id', cellData=blockIds,
                      fieldType=VTKWriter.VTKFieldType.SCALARS,
                      dataType=VTKWriter.VTKDataType.INT)
writer.write()

This is what we get when we open Paraview and visualize the `block_id` field. If you try it, you should see something similar.

![paraview shwoing blocks in mesh](blocks_elements.png)

Success! This is what were shooting for.

## Essential boundary conditions
We're going to make one more modification to the mesh. Looking again at the problem setup figure, we can see that we need to apply boundary conditions on the bottom, left, and top boundary edges of the slab. Similar to the `blocks` attribute, `mesh` has a `nodeSets` dictionary that maps names to sets of nodes. We'll make the nodesets we need by performing range queries over the nodal coordinates:

In [10]:
nodeTol = 1e-8
nodeSets = {'left': np.flatnonzero(mesh.coords[:,0] < xRange[0] + nodeTol),
            'right': np.flatnonzero(mesh.coords[:,0] > xRange[1] - nodeTol),
            'bottom': np.flatnonzero(mesh.coords[:,1] < yRange[0] + nodeTol),
            'top': np.flatnonzero(mesh.coords[:,1] > yRange[1] - nodeTol)}

# create a copy of the mesh that has the nodeSets in it
mesh = Mesh.mesh_with_nodesets(mesh, nodeSets)

Now we're going to register the essential boundary conditions so that the optimism solvers will know how to deal with them. This is done with an `EssentialBC` object. Each `EssentialBC` represents a boundary condition on one field component of a node set. As en example, let's create one to represent the $x$-component displacement boundary condition on the nodes of the left edge:

In [11]:
ebcLeft = Mesh.EssentialBC(nodeSet='left', field=0)

This is one boundary condition; we have three essential boundary conditions in total to apply. The thing to do is to collect them all in a list. So the code looks like it would in a real application, we'll ignore the `ebcLeft` we just created and create all of the essential boundary conditions in one statement:

In [12]:
EBCs = [Mesh.EssentialBC(nodeSet='left', field=0),
        Mesh.EssentialBC(nodeSet='bottom', field=1),
        Mesh.EssentialBC(nodeSet='top', field = 1)]

Next, we create a `DofManager` object. What is this for? It's a class to help us index into our nodal arrays, keeping track of which degrees of freedom have essential boundary conditions. It's purpose will become clearer when we use it later.

In [13]:
fieldShape = mesh.coords.shape
dofManager = Mesh.DofManager(mesh, fieldShape, EBCs)

The variable `fieldShape` tells `dofManager` what the array of the unknowns will look like. In solid mechanics, the unknown field is the displacement, which happen to have the same shape as the nodal coordinates, so the `mesh.coords` array is a convenient place to grab this information. You could also manually enter `(nNodes, 2)`, where `nNodes` is the total number of nodes in the mesh, and 2 is the number of components of displacement in plane strain.

We use the vis tools again to check that we've done the essential boundary condition specification correctly. First, we'll use a little trick to turn the boundary conditions into a viewable nodal field. We'll use an attribute of the `DofManager` class called `isBc`. This is just an array of size `fieldShape` that indicates whether each dof has an essential boundary condition. Numpy can cast this to an array of integers (more precisely, the `int64` type in numpy) with values 0 or 1 which can be plotted in Paraview. The `dataType` argument is different now; for block ID, it was a scalar field, but for boundary conditions, we want it to be a vector field (one component for each displacement component).

In [14]:
bcs = np.array(dofManager.isBc, dtype=np.int64)
writer.add_nodal_field(name='bcs', nodalData=bcs, fieldType=VTKWriter.VTKFieldType.VECTORS,
                       dataType=VTKWriter.VTKDataType.INT)
writer.write()

Note that `writer` still refers to the same `VTKWriter` object as before. A VTKWriter object is always associated with the same filename, so when we add a new field and then call the `write()` method, it will overwrite the previous VTK file. Indeed, if you open `check_problem_steup.vtk`, you'll see that it now contains two output fields, "block_id" and "bcs".

Contour plots of the $x$- and $y$-components of the "bcs" field are shown below:

![contour plots of boundary condition fields](boundary_conditions.jpeg)

The first plot shows all nodes with boundary conditions on the $x$ component of the displacement. We see that the left edge has a value of 1 (which means that the boundary condition flag is "True" there), and every other node has a value of 0, which means they are unrestrained. This is exactly what we want. The $y$ component plot also confirms that the top and bottom edges correctly have their boundary conditions. Of course, the top edge has an *inhomogeneous* boundary condition. We'll enforce the actual value of this boundary condition later.

## Build the function space
The next step is to create a representation of the discrete function space to help us do things like interpolate our fields and do calculus on them. The first ingredient we need is a quadrature rule. In optimism, quadrature rules are specified by the highest degree polynomial that they can exactly integrate. A smart way to do this is to choose it in relation to $p$, the polynomial degree of our interpolation.

In [15]:
p = mesh.masterElement.degree

In the linearized theory of solid mechanics, the natural trial/test function space is $H^1$, because the operator contains products of gradients of the displacement. Since our interpolation is of degree $p$, the displacement gradient is of degree $p-1$, and the inner product of gradients is of degree $2(p-1)$. Thus, we choose this as our quadrature rule precision to avoid under-integrating our operator:

In [16]:
# create the function space
quadRule = QuadratureRule.create_quadrature_rule_on_triangle(degree=2*(p - 1))

The benefit of specifying our quadrature rule in this way is that if we later decide to modify the mesh to use higher-order elements, the quadrature rule will be updated automatically. This helps keep us out of trouble with hourglassing problems. Note that our operator is *nonlinear*, so the quadrature rule won't integrate it exactly, but the accuracy requirement of the linearized theory is a good rule of thumb.

With the quadrature rule (and the mesh), we can now construct the function space:

In [17]:
fs = FunctionSpace.construct_function_space(mesh, quadRule)

We'll see this object in operation later when we set up our energy function.

## Material models
Next, we instantiate the material models for the slab. For the left side, we'll choose a Neo-Hookean material. Material models in optimism take their material parameters in the form of a dictionary. For Neo-Hookean, the required parameters are the elastic modulus and the Poisson's ratio (both taken about the undeformed state).

In [18]:
# create the material model for the left side
props = {'elastic modulus': 5.0,
         'poisson ratio': 0.25}
leftMaterialModel = Neohookean.create_material_model_functions(props)

TODO: The property names are not documented yet. For now, you can find them by inspecting the code in optimism/materials.

Now we'll instantiate the other material model for the right-hand side. We'll pick a $J_2$ plasticity model, which is a bit more interesting (and thus has more parameters).

In [19]:
E = 10.0
nu = 0.25
Y0 = 0.01*E
H = E/100
props = {'elastic modulus': E,
         'poisson ratio': nu,
         'yield strength': Y0,
         'hardening model': 'linear',
         'hardening modulus': H}
rightMaterialModel = J2Plastic.create_material_model_functions(props)


The meaning of the parameters is clear from the keys. There are several hardening models currently available, such as linear hardening, a version of power law hardening, and the Voce exponential saturation law. We'll keep it simple and just use linear hardening.

For multi-material simulations, we must create a dictionary of each material that maps the name of each element block to the material model object for that block, like this:

In [20]:
materialModels = {'left layer': leftMaterialModel, 'right layer': rightMaterialModel}


## Write the energy function to minimize

Numerical solutions to PDEs are obtained in optimism by minimizing an objective function, which may be thought of intuitively as en energy. In fact, for hyperelastic materials, the objective function *is* the total energy. For history-dependent materials, one can often write an incremental functional that is minimized by the displacement field that carries the body from one discrete time step to the next. We're going to write the function for our problem now.

There is a tool in the `Mechanics` module that will do most of the work for us. Let's call it first and explain its output afterwards.

In [29]:
    
# mechanics functions
mechanicsFunctions = Mechanics.create_multi_block_mechanics_functions(
    fs, mode2D="plane strain", materialModels=materialModels)


The `create_multi_block_mechanics_functions` writes some functions for us to help write the problem in the right form. The most important part of the output is `mechanicsFunctions.compute_strain_energy(...)`, which writes all the loops over the blocks, elements, and quadrature points that you would need to compute the energy from the nodal displacements. (Now we finally see the `FunctionSpace` object `fs` in action).
To use this new function, we'll invoke it like this:
```python
mechanicsFunctions.compute_strain_energy(U, internalVariables)
```
where `U` is the array of the degrees of freedom (the nodal displacements), and `internalVariables` is an array of the internal variables at every quadrature point. It has shape `(ne, neqp, nint)`, with `ne` being the number of elements, `neqp` the number of quadrature points per element, and `nint` the number of internal variables for the material model.  (NOTE: For multi-material simulations, this is currently sized such that `nint` is the *maximum* number of internal variables over all materials, so that the array is padded for materials using fewer internal variables. We may remove this restriction in the future to improve memory efficiency).

All of the minimization solvers in optimism require the objective function to have a signature like this:
```python
energy_function(Uu, p)
```
where `Uu` is the array of unknowns, and `p` is called the parameter set. The parameter set essentially holds all of the information that is needed to specify the problem but *isn't* the set of unknowns. These are things like values of the boundary conditions and the internal variables, as well as some other categories. Parameter sets are constructed by calling the `Params` function in the `Objective` module. This is to help organize them in certain categories that the solver needs to be aware of, such as which ones have derivatives taken with respect to inside the solver. We need to cast our energy function in this form. Let's write it like this and work out what the intervening steps must be:

```python
def energy_function(Uu, p):
    U = create_field(Uu, p)
    internalVariables = p.state_data
    return mechanicsFunctions.compute_strain_energy(U, internalVariables)
```
On the first line, we use the parameter set to pass from the array of unknowns to the full array of nodal displacements. This means we need to fill in the values of the inhomogeneous boundary conditions. Next, we pull out the internal variables from the parameter set. Finally, we use the canned `compute_strain_energy(...)` function with these variables in order to compute the total energy.

The inhomogeneous boundary condition part is handled like so:

In [34]:
# helper function to fill in nodal values of essential boundary conditions
def get_ubcs(p):
    appliedDisp = p[0]
    EbcIndex = (mesh.nodeSets['top'], 1)
    V = np.zeros(fieldShape).at[EbcIndex].set(appliedDisp)
    return dofManager.get_bc_values(V)



We will store the applied displacement in the first slot of the parameter set. In line 4 above we extract it. Then we make an array of the same size as the nodal displacements, set it to zero, and replace the values in the DOFs on the top edge with the value of the applied displacement.

Now we can write the `create_field` function shown above in the proposed `energy_function`:

In [36]:
# helper function to go from unknowns to full DoF array
def create_field(Uu, p):
    return dofManager.create_field(Uu, get_ubcs(p))
    


The pieces are finally in place to define the energy function for our problem:

In [37]:
# write the energy to minimize
def energy_function(Uu, p):
    U = create_field(Uu, p)
    internalVariables = p.state_data
    return mechanicsFunctions.compute_strain_energy(U, internalVariables)




## Set up the optimization solver
We have an objective function - `energy_function`, which we will hand to a routine that will find the unknowns displacements that minimize it. In this section, we specficy which optimization solver we want to use, and tell it how to work. 

We will use the Steighaug trust region method. This method uses linear conjugate gradient iterations as part of its algorithm, which in turn need to be preconditioned in order to be effective. Currently, the only available preconditioner in optimism is a Cholesky factorization on the stiffness matrix. We need to intruct the solver how to assemble the stiffness matrix like this:

In [38]:
# Tell objective how to assemble preconditioner matrix
def assemble_sparse_preconditioner_matrix(Uu, p):
    U = create_field(Uu, p)
    internalVariables = p.state_data
    elementStiffnesses = mechanicsFunctions.compute_element_stiffnesses(U, internalVariables)
    return SparseMatrixAssembler.assemble_sparse_stiffness_matrix(
        elementStiffnesses, mesh.conns, dofManager)


We see once again that `mechanicsFunctions` provides a helper function. In this case, the function `compute_element_stiffnesses` takes the same inputs as the energy function, but instead of returning the total energy, it returns an array containing the stiffness matrix for each element. The elemental stiffness matrices are the Hessians of the total energy in each element, and automatic differentiation is again used to perform this calculation. The `assemble_sparse_stiffness_matrix(...)` function takes these elemental stiffness matrices and contructs the system's stiffness matrix using a sparse matrix data structure (from SciPy). We tell the solver how to use this capability by building something called a `PrecondStrategy`: 

In [40]:
precondStrategy = Objective.PrecondStrategy(assemble_sparse_preconditioner_matrix)

# solver settings
solverSettings = EquationSolver.get_settings(use_preconditioned_inner_product_for_cg=True)


Finally, we can specify custom settings for the solver is we wish.

## Solve!

In [30]:
# initialize unknown displacements to zero
Uu = dofManager.get_unknown_values(np.zeros(fieldShape))

# set initial values of parameters
appliedDisp = 0.0
state = mechanicsFunctions.compute_initial_state()
p = Objective.Params(appliedDisp, state)
    
# Construct an objective object for the equation solver to work on
objective = Objective.ScaledObjective(energy_function, Uu, p, precondStrategy=precondStrategy)
    
# increment the applied displacement
appliedDisp = L*0.01
p = Objective.param_index_update(p, 0, appliedDisp)

# Find unknown displacements by minimizing the objective
Uu = EquationSolver.nonlinear_equation_solve(objective, Uu, p, solverSettings)



Assembling preconditioner 0
min, max diagonal stiffness =  0.9999999999999998 1.0000000000000002
Factorizing preconditioner
num warm start cg iters =  1
Assembling preconditioner 0
min, max diagonal stiffness =  0.9875365254464693 1.0110599625088337
Factorizing preconditioner

Initial objective, residual =  0.04170630570636782 1.5580526760827706e-05
obj= -1.47133479e-10 , model obj= -1.47130159e-10 , res=  5.10352716e-10 , model res=  7.43900588e-21 , cg iters=   1 , tr size=          2.0 ,  interior  , accepted=  True



## Post-process
The last step is to write out the simulation results so that we can use them. We've already seen a few examples of generating VTK output. Let's create another `VTKWriter` for the simulation results. Adding the displacement field is straightforward:

In [31]:
# write solution data to VTK file for post-processing
writer = VTKWriter.VTKWriter(mesh, baseFileName='uniaxial_two_material')

U = create_field(Uu, p)
writer.add_nodal_field(name='displacement', nodalData=U, fieldType=VTKWriter.VTKFieldType.VECTORS)



Let's also show an example of plotting quadrature field data. A commonly needed output is the stress field. We make use of another function in `mechanicsFunctions` to help us. However, before we write out any quadrature field data, we should update the internal variables. Currently, `state` still holds the initial conditions of the internal variables. That is, the solver finds the equilibrium displacement field, but doesn't change `state` in place. This is again due to Jax's functional programming paradigm. The following function call returns the updated internal variables using the new displacement field and the old internal variables as inputs:

In [ ]:
# update the state variables in the new equilibrium configuration
state = mechanicsFunctions.compute_updated_internal_variables(U, p.state_data)



We make use of another of the `mechanicsFunctions` member functions to get the stress field, using the updated internal variables. Note that we don't pay the cost of iteratively solving the history-dependent material model response again. This function expects that the internal variables are already updated when it is called, and simply uses theses values in the evaluation of the material model energy density functions.

In [ ]:
energyDensities, stresses = mechanicsFunctions.compute_output_energy_densities_and_stresses(
    U, state)


As the left-hand side of this assignment implies, the scalar energy density field (at every quadrature point) is returned in addition to the stresses. In fact, only the scalar-valued energy density function for each material is implemented in each material model. The stress field is the derivative of the energy density function with respect to the displacement gradient, and optimism uses Jax's automatic differentiation to compute this derivative. When computing this derivative, evaluating the energy density at the same time is essentially free, so the above function coputes them both.

The `stresses` array contains the stress tensor for every quadrature point of every element. There is no interpolation associated with a quadrature field, so in order to visualize the stresses, this must be remedied, either on the physics solver side or in the visualization software. One simple way to accomplish this is to compute element averages of the field and plot it as a cell-based field (instead of a nodal field). Optimism provides a method to do this:

In [ ]:
cellStresses = FunctionSpace.project_quadrature_field_to_element_field(fs, stresses)
writer.add_cell_field(name='stress', cellData=cellStresses,
                      fieldType=VTKWriter.VTKFieldType.TENSORS)

writer.write()




The above code can be wrapped in a loop that will compute the response at multiple time steps. Note that before moving to the next time step, you should update the internal variables in the parameter set:

In [33]:
# update the state variables in the new equilibrium configuration
p = Objective.param_index_update(p, 1, state)